<a href="https://colab.research.google.com/github/cassidyhanna/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/U2%2CS1%2CM2_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
# Change Column 'created' to datetime
# Make a df column to check the 'apartments' a studio 
# Make a df column to check if the apartments allow animals

df['created'] = pd.to_datetime(df['created'])
df['studio'] = (df['bedrooms'] < 1)
df['no_animals'] = (df['cats_allowed'] == 0 ) & (df['dogs_allowed'] == 0 )

df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# Change data type to float
df['bedrooms'] = df['bedrooms'].astype(np.float64)
df['bathrooms'] = df['bathrooms'].astype(np.float64)
df['price'] = df['price'].astype(np.float64)





In [3]:
train = df[df['created'] <= '2016-5-31' ]
test = df[df['created'] > '2016-05-31 ']

train.shape,test.shape

((31839, 36), (16978, 36))

In [4]:
#Choose features to predict with
features = ['bedrooms','bathrooms']
target = 'price'
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]


print(f'Linear Regression, dependent on: {features}')
print(X_train.shape, X_test.shape)

Linear Regression, dependent on: ['bedrooms', 'bathrooms']
(31839, 2) (16978, 2)


In [0]:
guess = y_train.mean()
baseline_y_train_pred = [guess] * len(y_train)
baseline_y_test_pred = [guess] * len(y_test)

In [6]:
train.head(5)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,studio,no_animals
2,1.0,1.0,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850.0,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
3,1.0,1.0,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275.0,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
4,1.0,4.0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350.0,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
5,2.0,4.0,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995.0,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
6,1.0,2.0,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600.0,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False


**Model predictions**

In [15]:

def predict(model,bedrooms, bathrooms ):
  main_model = model()
  main_model.fit(X_train,y_train)
  y_pred = main_model.predict([[bedrooms, bathrooms]])
  estimate = y_pred[0]
  #bedroom_coef, bathroom_coef = LR_model.coef_[0]
  results = f'${estimate:,.0f} is the estimated price to rent a {bathrooms:,.0f} bathroom, {bedrooms:,.0f} bedroom apartment in NY.'

  coefficients = pd.Series(main_model.coef_, features)
  result1 = coefficients.to_string() 
  result3 = f'Below is how much $$ each feature will add to the cost of the apartment'
  result2 = f'Intercept    {main_model.intercept_}'
  
  return print(results + '\n'  + result2 + '\n' + '\n' +  result3 + '\n' + result1   )

predict(LinearRegression,3,2)

$5,799 is the estimated price to rent a 2 bathroom, 3 bedroom apartment in NY.
Intercept    485.93860606945555

Below is how much $$ each feature will add to the cost of the apartment
bedrooms      389.491162
bathrooms    2072.306466


**Models Errors**

In [8]:
def metrics(model,X_train,y_train,X_test,y_test):
  
  #instantiate the model
  main_model = model()
  main_model.fit(X_train,y_train)
  
  #get prediction for X_train
  y_train_pred = main_model.predict(X_train)
  y_train_estimate = y_train_pred[0]
  
  #get prediction for X_test
  y_test_pred = main_model.predict(X_test)
  y_test_estimate = y_test_pred[0]
  
  #train predictions
  Train_mse = mean_squared_error(y_train, y_train_pred)
  Train_rmse = np.sqrt(Train_mse)
  Train_mae = mean_absolute_error(y_train, y_train_pred)
  Train_r2 = r2_score(y_train, y_train_pred)

  #test predictions
  Test_mse = mean_squared_error(y_test, y_test_pred)
  Test_rmse = np.sqrt(Test_mse)
  Test_mae = mean_absolute_error(y_test, y_test_pred)
  Test_r2 = r2_score(y_test, y_test_pred)

  #results
  #test_price = f'${y_test_estimate:,.0f} is the estimated price to rent this apartment in NY.' 
  #train_price = f'${y_train_estimate:,.0f} is the estimated price to rent this apartment in NY.'
  test_results = f'Heres the Test models, Mean Squared Error: {Test_mse:.2f}, Root Mean Squared Error:, {Test_rmse:.2f},Mean Absolute Error:,{Test_mae:.2f} and R^2:, {Test_r2:.2f} '
  train_results = f'Heres the Train models, Mean Squared Error: {Train_mse:.2f}, Root Mean Squared Error:, {Train_rmse:.2f},Mean Absolute Error:,{Train_mae:.2f} and R^2:, {Train_r2:.2f} '
  
  return print(test_results + '\n' + train_results)

metrics(LinearRegression,X_train,y_train,X_test,y_test)

Heres the Test models, Mean Squared Error: 1487509.91, Root Mean Squared Error:, 1219.64,Mean Absolute Error:,825.89 and R^2:, 0.52 
Heres the Train models, Mean Squared Error: 1517995.79, Root Mean Squared Error:, 1232.07,Mean Absolute Error:,818.55 and R^2:, 0.51 
